# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 12 2022 2:45PM,246187,22,627139,"NBTY Global, Inc.",Released
1,Sep 12 2022 2:37PM,246184,10,Enova-10625,Emerginnova,Released
2,Sep 12 2022 2:37PM,246184,10,Enova-10626,Emerginnova,Released
3,Sep 12 2022 2:36PM,246183,19,60017401,"GUSA Granules USA, Inc.",Released
4,Sep 12 2022 2:14PM,246181,10,8517406,RHL Supplies,Released
5,Sep 12 2022 2:06PM,246179,10,0085975582,ISDIN Corporation,Released
6,Sep 12 2022 2:06PM,246179,10,0085975581,ISDIN Corporation,Released
7,Sep 12 2022 2:06PM,246179,10,0085975585,ISDIN Corporation,Released
8,Sep 12 2022 2:06PM,246179,10,0085975587,ISDIN Corporation,Released
9,Sep 12 2022 2:06PM,246179,10,0085975586,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
23,246179,Released,26
24,246181,Released,1
25,246183,Released,1
26,246184,Released,2
27,246187,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
246179,NaN,NaN,26.0
246181,NaN,NaN,1.0
246183,NaN,NaN,1.0
246184,NaN,NaN,2.0
246187,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246097,0.0,6.0,12.0
246106,0.0,0.0,1.0
246127,0.0,0.0,2.0
246131,10.0,22.0,3.0
246135,0.0,2.0,4.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246097,0,6,12
246106,0,0,1
246127,0,0,2
246131,10,22,3
246135,0,2,4


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246097,0,6,12
1,246106,0,0,1
2,246127,0,0,2
3,246131,10,22,3
4,246135,0,2,4


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246097,,6,12
1,246106,,,1
2,246127,,,2
3,246131,10,22,3
4,246135,,2,4


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 12 2022 2:45PM,246187,22,"NBTY Global, Inc."
1,Sep 12 2022 2:37PM,246184,10,Emerginnova
3,Sep 12 2022 2:36PM,246183,19,"GUSA Granules USA, Inc."
4,Sep 12 2022 2:14PM,246181,10,RHL Supplies
5,Sep 12 2022 2:06PM,246179,10,ISDIN Corporation
31,Sep 12 2022 1:53PM,246176,10,Yusen – 3D Matrix
32,Sep 12 2022 1:50PM,246174,10,Yusen – 3D Matrix
33,Sep 12 2022 1:41PM,246097,15,"Person & Covey, Inc."
51,Sep 12 2022 1:39PM,246170,20,"Exact-Rx, Inc."
67,Sep 12 2022 1:38PM,246171,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 12 2022 2:45PM,246187,22,"NBTY Global, Inc.",,,1
1,Sep 12 2022 2:37PM,246184,10,Emerginnova,,,2
2,Sep 12 2022 2:36PM,246183,19,"GUSA Granules USA, Inc.",,,1
3,Sep 12 2022 2:14PM,246181,10,RHL Supplies,,,1
4,Sep 12 2022 2:06PM,246179,10,ISDIN Corporation,,,26
5,Sep 12 2022 1:53PM,246176,10,Yusen – 3D Matrix,,,1
6,Sep 12 2022 1:50PM,246174,10,Yusen – 3D Matrix,,,1
7,Sep 12 2022 1:41PM,246097,15,"Person & Covey, Inc.",,6,12
8,Sep 12 2022 1:39PM,246170,20,"Exact-Rx, Inc.",,8,8
9,Sep 12 2022 1:38PM,246171,10,ISDIN Corporation,,6,6


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 12 2022 2:45PM,246187,22,"NBTY Global, Inc.",1,,
1,Sep 12 2022 2:37PM,246184,10,Emerginnova,2,,
2,Sep 12 2022 2:36PM,246183,19,"GUSA Granules USA, Inc.",1,,
3,Sep 12 2022 2:14PM,246181,10,RHL Supplies,1,,
4,Sep 12 2022 2:06PM,246179,10,ISDIN Corporation,26,,
5,Sep 12 2022 1:53PM,246176,10,Yusen – 3D Matrix,1,,
6,Sep 12 2022 1:50PM,246174,10,Yusen – 3D Matrix,1,,
7,Sep 12 2022 1:41PM,246097,15,"Person & Covey, Inc.",12,6,
8,Sep 12 2022 1:39PM,246170,20,"Exact-Rx, Inc.",8,8,
9,Sep 12 2022 1:38PM,246171,10,ISDIN Corporation,6,6,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 12 2022 2:45PM,246187,22,"NBTY Global, Inc.",1,,
1,Sep 12 2022 2:37PM,246184,10,Emerginnova,2,,
2,Sep 12 2022 2:36PM,246183,19,"GUSA Granules USA, Inc.",1,,
3,Sep 12 2022 2:14PM,246181,10,RHL Supplies,1,,
4,Sep 12 2022 2:06PM,246179,10,ISDIN Corporation,26,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 12 2022 2:45PM,246187,22,"NBTY Global, Inc.",1.0,NaN,NaN
1,Sep 12 2022 2:37PM,246184,10,Emerginnova,2.0,NaN,NaN
2,Sep 12 2022 2:36PM,246183,19,"GUSA Granules USA, Inc.",1.0,NaN,NaN
3,Sep 12 2022 2:14PM,246181,10,RHL Supplies,1.0,NaN,NaN
4,Sep 12 2022 2:06PM,246179,10,ISDIN Corporation,26.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 12 2022 2:45PM,246187,22,"NBTY Global, Inc.",1.0,0.0,0.0
1,Sep 12 2022 2:37PM,246184,10,Emerginnova,2.0,0.0,0.0
2,Sep 12 2022 2:36PM,246183,19,"GUSA Granules USA, Inc.",1.0,0.0,0.0
3,Sep 12 2022 2:14PM,246181,10,RHL Supplies,1.0,0.0,0.0
4,Sep 12 2022 2:06PM,246179,10,ISDIN Corporation,26.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1969365,55.0,8.0,0.0
15,492224,14.0,6.0,0.0
16,246156,5.0,0.0,0.0
19,984647,22.0,2.0,19.0
20,738407,12.0,30.0,10.0
21,246147,0.0,1.0,0.0
22,246187,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1969365,55.0,8.0,0.0
1,15,492224,14.0,6.0,0.0
2,16,246156,5.0,0.0,0.0
3,19,984647,22.0,2.0,19.0
4,20,738407,12.0,30.0,10.0
5,21,246147,0.0,1.0,0.0
6,22,246187,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,55.0,8.0,0.0
1,15,14.0,6.0,0.0
2,16,5.0,0.0,0.0
3,19,22.0,2.0,19.0
4,20,12.0,30.0,10.0
5,21,0.0,1.0,0.0
6,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,55.0
1,15,Released,14.0
2,16,Released,5.0
3,19,Released,22.0
4,20,Released,12.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20,21,22
Status,,,,,,,
Completed,0.0,0.0,0.0,19.0,10.0,0.0,0.0
Executing,8.0,6.0,0.0,2.0,30.0,1.0,0.0
Released,55.0,14.0,5.0,22.0,12.0,0.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20,21,22
0,Completed,0.0,0.0,0.0,19.0,10.0,0.0,0.0
1,Executing,8.0,6.0,0.0,2.0,30.0,1.0,0.0
2,Released,55.0,14.0,5.0,22.0,12.0,0.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20,21,22
0,Completed,0.0,0.0,0.0,19.0,10.0,0.0,0.0
1,Executing,8.0,6.0,0.0,2.0,30.0,1.0,0.0
2,Released,55.0,14.0,5.0,22.0,12.0,0.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()